# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [29]:
%reload_ext dotenv
%dotenv

In [30]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [31]:
import os
from glob import glob

# Load the PRICE_DATA environment variable

PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))
parquet_files

['../../05_src/data/prices/CTAS/CTAS_2008.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2018.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2011.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2001.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2000.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2010.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2019.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2009.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2012.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2002.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2024.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2003.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2013.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2016.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2006.parquet/part.0.parqu

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [34]:
import numpy as np
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1),
                       Adj_Close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_Close_lag'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))


/var/folders/mn/hzb_sstd3t9f6xyvy2fry_5m0000gn/T/ipykernel_36839/3900174448.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [35]:
dd_feat.compute()

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
WELL,2007-01-03 00:00:00+00:00,18.323530,43.060001,43.060001,42.619999,43.000000,1119200,Real Estate,Health Care REITs,2007,NaN,NaN,NaN,0.440002
WELL,2007-01-04 00:00:00+00:00,18.527784,43.540001,43.540001,42.799999,43.099998,594200,Real Estate,Health Care REITs,2007,43.060001,18.323530,0.011147,0.740002
WELL,2007-01-05 00:00:00+00:00,18.532040,43.549999,43.970001,43.400002,43.549999,1119700,Real Estate,Health Care REITs,2007,43.540001,18.527784,0.000230,0.570000
WELL,2007-01-08 00:00:00+00:00,18.612896,43.740002,43.939999,43.400002,43.630001,361100,Real Estate,Health Care REITs,2007,43.549999,18.532040,0.004363,0.539997
WELL,2007-01-09 00:00:00+00:00,18.710766,43.970001,43.990002,43.599998,43.900002,465600,Real Estate,Health Care REITs,2007,43.740002,18.612896,0.005258,0.390003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AMP,2024-10-21 00:00:00+00:00,522.640015,522.640015,523.880005,519.609985,522.760010,360400,Financials,Asset Management & Custody Banks,2024,523.159973,523.159973,-0.000994,4.270020
AMP,2024-10-22 00:00:00+00:00,519.559998,519.559998,521.729980,516.950012,520.000000,338600,Financials,Asset Management & Custody Banks,2024,522.640015,522.640015,-0.005893,4.779968
AMP,2024-10-23 00:00:00+00:00,520.469971,520.469971,523.419983,517.590027,518.400024,387800,Financials,Asset Management & Custody Banks,2024,519.559998,519.559998,0.001751,5.829956


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [36]:
# Convert to pandas DataFrame
pd_feat = dd_feat.compute()

# Add rolling average return calculation with a window of 10 days
pd_feat['rolling_avg_return'] = pd_feat['returns'].rolling(window=10).mean()

# Display the result
print(pd_feat.tail())

Price                       Date   Adj Close       Close        High  \
ticker                                                                 
AMP    2024-10-21 00:00:00+00:00  522.640015  522.640015  523.880005   
AMP    2024-10-22 00:00:00+00:00  519.559998  519.559998  521.729980   
AMP    2024-10-23 00:00:00+00:00  520.469971  520.469971  523.419983   
AMP    2024-10-24 00:00:00+00:00  504.209991  504.209991  506.730011   
AMP    2024-10-25 00:00:00+00:00  504.480011  504.480011  507.089996   

Price          Low        Open  Volume      sector  \
ticker                                               
AMP     519.609985  522.760010  360400  Financials   
AMP     516.950012  520.000000  338600  Financials   
AMP     517.590027  518.400024  387800  Financials   
AMP     487.130005  500.540009  726500  Financials   
AMP     500.589996  506.950012  430000  Financials   

Price                          subsector  year   Close_lag  Adj_Close_lag  \
ticker                                 

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

- It was not necessary to convert to pandas to calculate the moving average return.

- We could perform the rolling average return calculation using Dask directly, without converting to a Pandas DataFrame.

- Calculate rolling average return with a window of 10 days using Dask:

- Dask is designed to handle datasets that are too large to fit into memory, leveraging parallel computation across multiple cores. Using Dask for tasks like rolling calculations not only avoids unnecessary data transfers between Pandas and Dask but also ensures optimal performance and scalability for large-scale data processing.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.